## Script 
This script reads a input file name "input.dat" and returns top 5 matching data from the knowledge graph. 

## The knowledge graph 
It is hosted in the graph database neo4j. With py2neo, a package which helps to connect neo4j with python, we connect to the database. We run the query to return the most matching words from the list of query words named "input.dat". Before running this script, you must have loaded the knowledge graph(db.csv) in the neo4j graph and should be running. The instructions for loading the knowledge graph is mentioned in the README.md file in github. 

## Format for input.dat
 The file must contain query words seperated by commas. And the query words must be contained in the graph. For eg:
 fawn,pet

In [129]:
import time
import pandas as pd
from py2neo import Graph, Node
graph = Graph(password = "rosebay")

In [130]:
# Read the input query words from the dat file
query_words = [word.split(',') for word in open("input.dat","r").readlines()][0]
query_words

['captain', 'chair']

## Recursive Neighbor algorithm based on Breadth First Search

### Finding the first neighbors for each query word  

In [131]:
d = {}
'''
The dictionary d will store the neighbor for each query node and the distances between them.
'''
for each in query_words:
    query1 = '''
    MATCH (n:Word)-[r]->(n2:Word) where n.name= '%s' RETURN n2.name as words,r.weight as %s order by %s asc
    '''%(each,each,each)
    data = graph.run(query1).data()
    d[each]= pd.DataFrame(data)
    

In [132]:
'''
Concat the distance for each node for creating an adjacency matrix with query words as columns, its neighbor as rows.
'''
total=pd.concat(d.values(),axis=0,sort=False)
total.head(2)

,captain,words,chair
0,3.0,angry,NaN
1,4.0,ring,NaN


In [133]:
'''
Groupby operation is performed to combine the distance of each neighbor to query words.
Missing connection between nodes are replaced with a distance of 1000.
'''
merged_total=total.groupby(by=['words']).agg('sum')
merged_total.replace(0,1000,inplace=True)
merged_total.head(2)

,captain,chair
words,,
angry,3.0,1000.0
bed,1000.0,7.0


In [134]:
'''
Total distance is calculated by add the distance of neighbor with both query words. And the words are kept in a sorted order.
'''
merged_total['total']=merged_total.sum(axis=1)
merged_total.sort_values('total',inplace=True)
merged_total.head(5)

,captain,chair,total
words,,,
pilot,13.0,7.0,20.0
angry,3.0,1000.0,1003.0
smile,4.0,1000.0,1004.0
shake,4.0,1000.0,1004.0
ring,4.0,1000.0,1004.0


In [135]:
'''
A global list is generated storing the top 5 closed neighbors with the query node
'''
global_list =merged_total[:5]
global_list.head()

,captain,chair,total
words,,,
pilot,13.0,7.0,20.0
angry,3.0,1000.0,1003.0
smile,4.0,1000.0,1004.0
shake,4.0,1000.0,1004.0
ring,4.0,1000.0,1004.0


#### Expanding the hops
At first, Queue is filled with first neighbor or global_list 
Steps:
    1. Start exploring each node of the queue.
    2. Generate distance for neighbors of that node with every query words.
    3. For each neighbor, if the total distance[collective distance] is less than max, than add that to the queue. (As adding          nodes having less distance than max can expand to more close node)
    4. Update the global list with new neighbour and also update the max_distance
    5. Repeat until a max_limit or queue is empty

In [136]:
# get maximum distance
max_distance = global_list['total'].iloc[4]
max_distance

1004.0

In [137]:
# Create a list of neighbor to explore
queue = [row_index for row_index,row in global_list.iterrows()]
queue

['pilot', 'angry', 'smile', 'shake', 'ring']

In [138]:
# Create a list of explored nodes
explored = list()

In [139]:
i=0
while len(queue)!=0:
    i+=1
    '''
    Set a limit of maximum node to expand(optional). Can be removed
    '''
    if i >=30:
        break
    each_word = queue.pop(0)
    d2 = {}
    print("--------------------------------------")    
    print("Expanding node %s:"%each_word)
    print("explored nodes: ")
    print(explored)
    print("nodes in queue: ")
    print(queue)
    print("--------------------------------------")
    print("max distance is: %d"%max_distance)
    '''
    Create a table for each query_words in dictionary and at last, concat all the tables
    '''
    for each in query_words:
        query='''// neighbor nodes and total distance to a query word
        match(n:Word)-[r]->(n1:Word)-[r2]->(n2:Word) 
        where n1.name='%s' AND n.name='%s'
        return n2.name as words,sum(r.weight+r2.weight) as %s
        order by words,%s'''%(each_word,each,each,each)
        data2 = graph.run(query).data()
        d2[each]=pd.DataFrame(data2)
    aggregate=pd.concat(d2.values(),axis=0,sort=False) 
    #TODO: Discard empty dataframes
    if aggregate.shape[0]==0:   
        explored.append(each_word)
        continue
    # Remove rows including query values
    aggregate[~aggregate['words'].isin(query_words)]

    aggregate_total=aggregate.groupby(by=['words']).agg('sum')
    
    aggregate_total.replace(0,1000,inplace=True)
    #check if columns are missing
    A = set(aggregate_total)
    B = set(query_words)
    missing=list(B-A)
    for each in missing:
        aggregate_total[each]= 1000
    aggregate_total['total']=aggregate_total.sum(axis=1)
    #print(aggregate_total.head(2))
    aggregate_total.sort_values('total',inplace=True)    
    # check if the node is explored and distance is less than max for adding to the queue
    less_than_max=aggregate_total.index[aggregate_total['total'] < max_distance].tolist()
    for each in less_than_max:
        if each not in explored:
            queue.append(each)
    global_list=global_list.append(aggregate_total,sort=True)
    global_list.sort_values('total',inplace=True)
    # update max
    max_distance = global_list['total'].iloc[4]
    explored.append(each_word)
    

--------------------------------------
Expanding node pilot:
explored nodes: 
[]
nodes in queue: 
['angry', 'smile', 'shake', 'ring']
--------------------------------------
max distance is: 1004
--------------------------------------
Expanding node angry:
explored nodes: 
['pilot']
nodes in queue: 
['smile', 'shake', 'ring', 'father', 'plant', 'down', 'helicopter', 'whale', 'wing', 'fly', 'plane', 'train']
--------------------------------------
max distance is: 41
--------------------------------------
Expanding node smile:
explored nodes: 
['pilot', 'angry']
nodes in queue: 
['shake', 'ring', 'father', 'plant', 'down', 'helicopter', 'whale', 'wing', 'fly', 'plane', 'train']
--------------------------------------
max distance is: 41
--------------------------------------
Expanding node shake:
explored nodes: 
['pilot', 'angry', 'smile']
nodes in queue: 
['ring', 'father', 'plant', 'down', 'helicopter', 'whale', 'wing', 'fly', 'plane', 'train']
--------------------------------------
max

In [140]:
global_list.sort_values('total',inplace=True)
global_list.head(10)

,captain,chair,total
words,,,
pilot,13.0,7.0,20.0
father,25.0,13.0,38.0
plant,25.0,13.0,38.0
down,27.0,14.0,41.0
helicopter,27.0,14.0,41.0
whale,27.0,14.0,41.0
wing,27.0,14.0,41.0
fly,29.0,15.0,44.0
plane,31.0,16.0,47.0


In [141]:
# Generate csv for writing the result. Uncomment the code below to write

#global_list.to_csv('result.csv')

## Minimum Spanning Tree

In [142]:
# Read input from the dat file
words = [word.split(',') for word in open("input.dat","r").readlines()][0]

In [143]:
words

['captain', 'chair']

In [144]:
'''
create a Minimum Spanning Tree
'''

query2 = '''// MST
match (n:Word) where n.name in ['captain','chair']
call algo.spanningTree.minimum('Word','RELATED','weight',id(n),{write:true, writeProperty:"MINST"})
yield effectiveNodeCount
return effectiveNodeCount;'''

In [145]:
'''
Read the MST
'''

query2='''// Read MST Relationship
match path = (n:Word)-[:MINST]->() where n.name in ['sofa','fawn']
with relationships(path) as rels
unwind rels as rel
with distinct rel as rel
Return startNode(rel).name as source, endNode(rel).name as destination, rel.weight as cost'''